# **Introdução aos Modelos DSGE**

João Ricardo Costa Filho \\
https://www.joaocostafilho.com

# **Produção Domiciliar e Ciclos de Negócio Reais (RBC)**

## Instale na máquina "virtual" o Octave e o Dynare.

In [ ]:
!sudo apt-get update
!sudo apt-get install -y x13as

In [2]:
!apt-get install octave &> /dev/null

In [3]:
!apt-get install dynare &> /dev/null

### Crie o arquivo .mod

In [9]:
%%writefile rbc_HP.mod

@#define antecipacao=0

var c, cm, ch, y, km, kh, hm, hh, w, l, r, xm, xh, x, zm, zh, G;

varexo em, eh, eg;

parameters beta, th, tk, deltam, deltah, alpha, eta, hmbar, hhbar, rhom, rhoh, rhog, e, gs;

beta   = 0.97;
th     = 0.34;
tk     = 0.25;
deltam = 0.05;
deltah = 0.05;
alpha  = 0.44;
eta    = 0.3245;
hmbar  = 0.33;
hhbar  = 0.25;
rhom   = 0.9;
rhoh   = 0.9;
rhog   = 0.8;
e      = 2 / 3;
gs     = 0.2;


model(linear);

# rbar  = ( ( 1 / beta ) + deltam - 1 - tk * deltam ) / ( 1 - tk );
# kmbar = hmbar * ( rbar / alpha )^( 1 / ( alpha - 1 ) );
# ybar  = kmbar^alpha * hmbar^( 1 - alpha );
# xmbar = deltam * kmbar;
# wbar  = ( 1 - alpha ) * ( ybar / hmbar );
# lbar  = 1 - hmbar - hhbar;
# khbar = beta * hhbar * wbar * ( 1 - th) / ( 1 - beta * ( 1 - deltah ) * ( 1 - eta ) );
# chbar = khbar^eta * hhbar^( 1 - eta );
# xhbar = deltah * khbar;
# xbar  = xmbar + xhbar;
# Gbar  = gs * ybar;
# cmbar = ybar - xbar - Gbar;
#   phi = ( lbar * ( cmbar^(e-1) ) * wbar * ( 1 - th ) )^(-1); % variável auxiliar
# kappa = hhbar * ( lbar * ( chbar^e ) * ( 1 - eta ) )^(-1);   % variável auxiliar
#     a = phi / ( kappa + phi );
#  cbar = ( a * cmbar^e + ( 1 - a ) * chbar^e )^( 1 / e );
#     b = phi / ( phi + a * cbar^(-e) );
#   chi = ( 1 - tk ) / ( rbar * ( 1 - tk ) + tk * deltam + ( 1 - deltam ) );  % variável auxiliar
#     g = ( a * ( e - 1 ) * cmbar^( e - 1 ) * ( 1 - deltah ) ) / ( a *cmbar^( e - 1 ) *( 1 - deltah ) + ( 1 - a ) * chbar^e * eta / khbar ); % variável auxiliar
#     q = ( ( 1 - a ) * e * ( 1 - eta ) * ( 1 / khbar ) * chbar^e ) / ( a * cmbar^( e - 1 )*( 1 - deltah ) + ( 1 - a ) * chbar^e * eta / khbar );  % variável auxiliar

e * c + ( 1 - e ) * cm - w = l;

e * ch - e * c = hh - l;

( e - 1 ) * cm(+1) - e * c(+1) + chi * rbar * r(+1) = (e-1) * cm - e * c;

e * c + ( 1 - e ) * cm = e * c(+1) - g * cm(+1) - e * q * ch(+1) + q * kh;

c = ( cbar^-e ) * ( a * ( cmbar^e ) * cm + ( 1 - a ) * ( chbar^e ) * ch );

ch = eta * kh(-1) + ( 1 - eta ) * zh + ( 1 - eta ) * hh;

xmbar * xm = kmbar * ( km - ( 1 - deltam ) * km(-1) );

xhbar * xh = khbar * ( kh - ( 1 - deltah ) * kh(-1) );

y = alpha * km(-1) + ( 1 - alpha ) * zm + ( 1 - alpha) * hm;

y - km(-1) = r;

y - hm = w;

@#if antecipacao==0
    G = rhog * G(-1) + eg;
@#else
    G = rhog * G(-1) + eg(-3);
@#endif

ybar * y = cmbar * cm + xbar * x + Gbar * G;

xbar * x = xmbar * xm + xhbar * xh;

lbar * l = - ( hmbar * hm + hhbar * hh );

zm = rhom * zm(-1) + em;

zh = rhoh * zh(-1) + eh;

end;

steady;

check;

shocks;

var eg;
stderr 1;
end;

stoch_simul(ar=1, order=1, irf=20);


Overwriting rbc_HP.mod


In [10]:
%%writefile solve.m
dynare rbc_HP.mod

Overwriting solve.m


In [11]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 17 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 2).
Processing outputs ...
done
Preprocessing completed.


STEADY-STATE RESULTS:

c  		 0
cm 		 0
ch 		 0
y  		 0
km 		 0
kh 		 0
hm 		 0
hh 		 0
w  		 0
l  		 0
r  		 0
xm 		 0
xh 		 0
x  		 0
zm 		 0
zh 		 0
G  		 0

EIGENVALUES:
         Modulus             Real        Imaginary

       1.642e-15       -1.642e-15                0
             0.8              0.8                0
             0.9              0.9                0
             0.9              0.9                0
          0.9399           0.9399                0
           1.124            1.124                0
       2.555e+16       -2.555e+16                0
       4.133e+16       -4.